In [ ]:
%cd /content/drive/MyDrive/OpenAi/Scanner
!git clone https://github.com/daveshap/Quickly_Extract_Science_Papers.git

/content/drive/MyDrive/OpenAi/Scanner
Cloning into 'Quickly_Extract_Science_Papers'...
remote: Enumerating objects: 65, done.
remote: Counting objects: 100% (12/12), done.
remote: Compressing objects: 100% (6/6), done.
remote: Total 65 (delta 9), reused 6 (delta 6), pack-reused 53
Receiving objects: 100% (65/65), 29.73 MiB | 18.85 MiB/s, done.
Resolving deltas: 100% (15/15), done.


In [ ]:
%cd /content/drive/MyDrive/OpenAi/Scanner/Quickly_Extract_Science_Papers
!pip install -r requirements.txt

In [ ]:
#@title Chat.py
import openai
from time import time, sleep
from halo import Halo
import textwrap
import yaml


###     file operations


def save_file(filepath, content):
    with open(filepath, 'w', encoding='utf-8') as outfile:
        outfile.write(content)



def open_file(filepath):
    with open(filepath, 'r', encoding='utf-8', errors='ignore') as infile:
        return infile.read()


def save_yaml(filepath, data):
    with open(filepath, 'w', encoding='utf-8') as file:
        yaml.dump(data, file, allow_unicode=True)


def open_yaml(filepath):
    with open(filepath, 'r', encoding='utf-8') as file:
        data = yaml.load(file, Loader=yaml.FullLoader)
    return data


###     API functions


def chatbot(conversation, model="gpt-3.5-turbo-16k-0613", temperature=0):#gpt-4-0613
    max_retry = 7
    retry = 0
    while True:
        try:
            spinner = Halo(text='Thinking...', spinner='dots')
            spinner.start()

            response = openai.ChatCompletion.create(model=model, messages=conversation, temperature=temperature)
            text = response['choices'][0]['message']['content']

            spinner.stop()

            return text, response['usage']['total_tokens']
        except Exception as oops:
            print(f'\n\nError communicating with OpenAI: "{oops}"')
            if 'maximum context length' in str(oops):
                a = conversation.pop(0)
                print('\n\n DEBUG: Trimming oldest message')
                continue
            retry += 1
            if retry >= max_retry:
                print(f"\n\nExiting due to excessive errors in API: {oops}")
                exit(1)
            print(f'\n\nRetrying in {2 ** (retry - 1) * 5} seconds...')
            sleep(2 ** (retry - 1) * 5)


def chat_print(text):
    formatted_lines = [textwrap.fill(line, width=120, initial_indent='    ', subsequent_indent='    ') for line in text.split('\n')]
    formatted_text = '\n'.join(formatted_lines)
    print('\n\n\nCHATBOT:\n\n%s' % formatted_text)




if __name__ == '__main__':
    openai.api_key = open_file('key_openai.txt').strip()
    paper = open_file('input.txt')
    if len(paper) > 22000:
        paper = paper[0:22000]
    ALL_MESSAGES = [{'role':'system', 'content': paper}]
    while True:
        # get user input
        text = input('\n\n\nUSER:\n\n')
        if text == '':
            # empty submission, probably on accident
            continue
        ALL_MESSAGES.append({'role': 'user', 'content': text})

        # get response
        response, tokens = chatbot(ALL_MESSAGES)
        if tokens >= 7800:
            a = ALL_MESSAGES.pop(1)
        chat_print(response)
        ALL_MESSAGES.append({'role': 'assistant', 'content': response})

In [ ]:
#@title Generate multiple reports
import openai
from time import time, sleep
from halo import Halo
import textwrap
import yaml
import os
import PyPDF2


prompts = [
'Can you give me a very clear explanation of the core assertions, implications, and mechanics elucidated in this paper?',

"Can you explain the value of this in basic terms? Like you're talking to a CEO. So what? What's the bottom line here?",

'Can you give me an analogy or metaphor that will help explain this to a broad audience.',
]



###     file operations


def save_file(filepath, content):
    with open(filepath, 'w', encoding='utf-8') as outfile:
        outfile.write(content)



def open_file(filepath):
    with open(filepath, 'r', encoding='utf-8', errors='ignore') as infile:
        return infile.read()


def save_yaml(filepath, data):
    with open(filepath, 'w', encoding='utf-8') as file:
        yaml.dump(data, file, allow_unicode=True)


def open_yaml(filepath):
    with open(filepath, 'r', encoding='utf-8') as file:
        data = yaml.load(file, Loader=yaml.FullLoader)
    return data


###     API functions


def chatbot(conversation, model="gpt-3.5-turbo-16k-0613", temperature=0):
    max_retry = 7
    retry = 0
    while True:
        try:
            spinner = Halo(text='Thinking...', spinner='dots')
            spinner.start()

            response = openai.ChatCompletion.create(model=model, messages=conversation, temperature=temperature)
            text = response['choices'][0]['message']['content']

            spinner.stop()

            return text, response['usage']['total_tokens']
        except Exception as oops:
            print(f'\n\nError communicating with OpenAI: "{oops}"')
            if 'maximum context length' in str(oops):
                a = conversation.pop(0)
                print('\n\n DEBUG: Trimming oldest message')
                continue
            retry += 1
            if retry >= max_retry:
                print(f"\n\nExiting due to excessive errors in API: {oops}")
                exit(1)
            print(f'\n\nRetrying in {2 ** (retry - 1) * 5} seconds...')
            sleep(2 ** (retry - 1) * 5)



def chat_print(text):
    formatted_lines = [textwrap.fill(line, width=120, initial_indent='    ', subsequent_indent='    ') for line in text.split('\n')]
    formatted_text = '\n'.join(formatted_lines)
    print('\n\n\nCHATBOT:\n\n%s' % formatted_text)




if __name__ == '__main__':
    # instantiate chatbot, variables
    openai.api_key = open_file('key_openai.txt').strip()

    # Get list of all PDF files in the input folder
    pdf_files = [f for f in os.listdir('input/') if f.endswith('.pdf')]

    for pdf_file in pdf_files:
        # Check if the report already exists in the output folder
        filename = 'output/' + pdf_file.replace('.pdf', '.txt')
        if os.path.exists(filename):
            continue

        # Open the PDF file
        with open('input/' + pdf_file, 'rb') as file:
            pdf_reader = PyPDF2.PdfReader(file)
            paper = ''
            for page_num in list(range(0,len(pdf_reader.pages))):
                page = pdf_reader.pages[page_num]
                paper += page.extract_text()

        if len(paper) > 22000:
            paper = paper[0:22000]
        ALL_MESSAGES = [{'role':'system', 'content': paper}]
        report = ''
        for p in prompts:
            ALL_MESSAGES.append({'role':'user', 'content': p})
            response, tokens = chatbot(ALL_MESSAGES)
            chat_print(response)
            ALL_MESSAGES.append({'role':'assistant', 'content': response})
            report += '\n\n\n\nQ: %s\n\nA: %s' % (p, response)

        # Save the report in the output folder with the same name as the PDF file but with .txt extension
        save_file(filename, report.strip())

In [ ]:
'''Repo Contents
chat.py - this file is a simple chatbot that will chat with you about the contents of input.txt (you can copy/paste anything into this text file). Very useful to quickly discuss papers.
generate_multiple_reports.py - this will consume all PDFs in the input/ folder and generate summaries in the output/ folder. This is helpful for bulk processing such as for literature reviews.
render_report.py - this will render all the reports in output/ to a an easier to read file in report.html.
EXECUTIVE SUMMARY
This repository contains Python scripts that automate the process of generating reports from PDF files using OpenAI's GPT-4 model. The scripts extract text from PDF files, send the text to the GPT-4 model for processing, and save the generated reports as text files. The scripts also include functionality to render the generated reports as an HTML document for easy viewing.

SETUP
Clone the repository to your local machine.
Install the required Python packages by running pip install -r requirements.txt in your terminal.
Obtain an API key from OpenAI and save it in a file named key_openai.txt in the root directory of the repository.
Place the PDF files you want to generate reports from in the input/ directory.
USAGE
Run the generate_multiple_reports.py script to generate reports from the PDF files in the input/ directory. The generated reports will be saved as text files in the output/ directory.
Run the render_report.py script to render the generated reports as an HTML document. The HTML document will be saved as report.html in the root directory of the repository.
You can modify the prompts in generate_multiple_reports.py to focus on any questions you would like to ask. In other words you can automatically ask any set of questions in bulk against any set of papers. This can help you greatly accelerate your literature reviews and surveys.
NOTE
The scripts are designed to handle errors and retries when communicating with the OpenAI API. If the API returns an error due to the maximum context length being exceeded, the scripts will automatically trim the oldest message and retry the API call. If the API returns any other type of error, the scripts will retry the API call after a delay, with the delay increasing exponentially for each consecutive error. If the API returns errors for seven consecutive attempts, the scripts will stop and exit.'''